# Предобработка и внешние фичи

In [1]:
from pathlib import Path
import json
import numpy as np
import pandas as pd

In [2]:
PROJECT_ROOT = Path.cwd()
candidate_event_dirs = [
    PROJECT_ROOT / 'data/marketplace/events',
    PROJECT_ROOT / 'data/events',
    PROJECT_ROOT / 'data copy/marketplace/events',
]
EVENTS_DIR = next((p for p in candidate_event_dirs if p.exists()), candidate_event_dirs[0])
USERS_PATH = PROJECT_ROOT / 'data/users.pq'
PRODUCTS_PATH = PROJECT_ROOT / 'psb_products_updated.json'

N_FILES = 2  
ANCHOR_DATE = pd.Timestamp('2023-01-01')  # надо понять
print('Используем каталог событий:', EVENTS_DIR)

Используем каталог событий: /home/mariyasoloveva/AVIAHACKATHON2025/data/marketplace/events


In [3]:
if EVENTS_DIR.exists():
    event_files = sorted(EVENTS_DIR.glob('*.pq'))
else:
    event_files = []
print(f'Файлов: {len(event_files)}')

Файлов: 1


In [4]:
if event_files:
    use_files = event_files if N_FILES is None else event_files[:N_FILES]
    raw_frames = [pd.read_parquet(f) for f in use_files]
    raw_events = pd.concat(raw_frames, ignore_index=True)
else:
    use_files = []
    raw_events = pd.DataFrame()
print(f'Загружено {len(raw_events):,} строк из {len(use_files)} файлов')
raw_events.head()

Загружено 2,942,392 строк из 1 файлов


,timestamp,user_id,item_id,subdomain,action_type,os
0,1000 days 00:00:00.062127,15549980,nfmcg_25621580,search,view,android
1,1000 days 00:00:00.116202,34952676,nfmcg_25968387,u2i,view,ios
2,1000 days 00:00:00.159632,43419595,nfmcg_25497204,u2i,view,android
3,1000 days 00:00:00.215206,58440865,nfmcg_7658098,search,view,android
4,1000 days 00:00:00.258081,70296647,nfmcg_8361059,catalog,view,android


# Убираем view

In [5]:
rows_before = len(raw_events)
if 'action_type' in raw_events.columns:
    print('action_type топ-значения ДО фильтрации:')
    print(raw_events['action_type'].value_counts().head())
    events_filtered = raw_events[raw_events['action_type'].fillna('') != 'view'].copy()
else:
    events_filtered = raw_events.copy()
rows_after = len(events_filtered)
print(f'Строк до: {rows_before:,} / после удаления view: {rows_after:,}')
if 'action_type' in events_filtered.columns:
    print('action_type топ-значения ПОСЛЕ фильтрации:')
    print(events_filtered['action_type'].value_counts().head())
events_filtered.head()

action_type топ-значения ДО фильтрации:
action_type
view        2809491
click        114652
clickout      14901
like           3348
Name: count, dtype: int64
Строк до: 2,942,392 / после удаления view: 132,901
action_type топ-значения ПОСЛЕ фильтрации:
action_type
click       114652
clickout     14901
like          3348
Name: count, dtype: int64


,timestamp,user_id,item_id,subdomain,action_type,os
61,1000 days 00:00:02.768145,10293544,nfmcg_10315542,u2i,click,android
146,1000 days 00:00:07.356507,26642308,nfmcg_14408646,u2i,click,android
165,1000 days 00:00:07.941704,1702253,nfmcg_1014623,u2i,click,android
167,1000 days 00:00:07.941704,42496109,nfmcg_24988397,u2i,click,ios
168,1000 days 00:00:07.941704,51327915,nfmcg_27391300,u2i,click,android


# Нормально приводим время

In [6]:
def choose_first_available(df: pd.DataFrame, candidates):
    for col in candidates:
        if col in df.columns:
            return col
    return None

In [7]:
def standardize_event_time(df: pd.DataFrame, anchor_date: pd.Timestamp) -> pd.DataFrame:
    df = df.copy()
    if df.empty:
        df['event_dt'] = pd.NaT
        df['date'] = pd.NaT
        return df
    time_col = choose_first_available(df, ['event_time', 'timestamp', 'time', 'ts'])
    if time_col is None:
        raise ValueError('Не нашли колонку со временем события')
    raw = df[time_col]
    if np.issubdtype(raw.dtype, np.timedelta64):
        event_dt = anchor_date + pd.to_timedelta(raw)
    else:
        event_dt = pd.to_datetime(raw, errors='coerce', utc=True)
        try:
            event_dt = event_dt.dt.tz_convert(None)
        except TypeError:
            pass
    df['event_dt'] = event_dt
    df['date'] = df['event_dt'].dt.floor('D')
    return df


In [8]:
events = standardize_event_time(events_filtered, ANCHOR_DATE)
if events.empty:
    print('Нет событий после фильтрации, дальнейшие шаги дадут пустые фичи')
else:
    events = events.dropna(subset=['event_dt'])

if 'user_id' not in events.columns and not events.empty:
    raise ValueError('Нет колонки user_id')

if 'user_id' in events.columns:
    events['user_id'] = pd.to_numeric(events['user_id'], errors='coerce').astype('Int64')
    events = events.dropna(subset=['user_id'])
    events['user_id'] = events['user_id'].astype(int)
print('После нормализации времени и user_id:', events.shape)
events[['event_dt', 'user_id']].head()

После нормализации времени и user_id: (132901, 8)


,event_dt,user_id
61,2025-09-27 00:00:02.768145,10293544
146,2025-09-27 00:00:07.356507,26642308
165,2025-09-27 00:00:07.941704,1702253
167,2025-09-27 00:00:07.941704,42496109
168,2025-09-27 00:00:07.941704,51327915


# Сумма по событию

In [9]:
amount_col = choose_first_available(events, ['price', 'amount', 'sum', 'value'])
if amount_col:
    events['amount'] = pd.to_numeric(events[amount_col], errors='coerce').fillna(0.0)
else:
    events['amount'] = 0.0

# главный признак категории/субдомена для группировок
category_col = choose_first_available(events, ['category', 'category_id', 'subdomain', 'domain', 'brand_id'])
if category_col:
    events[category_col] = events[category_col].astype(str)

print('Колонка суммы:', amount_col)
print('Колонка категории:', category_col)
preview_cols = ['amount'] + ([category_col] if category_col else [])
events[preview_cols].head()

Колонка суммы: None
Колонка категории: subdomain


,amount,subdomain
61,0.0,u2i
146,0.0,u2i
165,0.0,u2i
167,0.0,u2i
168,0.0,u2i


In [10]:
def detect_keyword_flag(df: pd.DataFrame, columns, keywords) -> pd.Series:
    cols = [c for c in columns if c and c in df.columns]
    if not cols:
        return pd.Series(False, index=df.index)
    text = df[cols[0]].fillna('').astype(str)
    for col in cols[1:]:
        text = text.str.cat(' ' + df[col].fillna('').astype(str))
    text = text.str.lower()
    mask = pd.Series(False, index=df.index)
    for kw in keywords:
        mask = mask | text.str.contains(kw)
    return mask

In [11]:
# Флаги по ключевым слова
category_candidates = [category_col, 'brand_id', 'domain', 'subdomain', 'item_id', 'action_type']
auto_keywords = ['auto', 'car', 'fuel', 'gas', 'azs', 'sto', 'parking', 'tire', 'taxi']
home_keywords = ['home', 'repair', 'remont', 'stroi', 'furniture', 'kitchen', 'flat', 'rent', 'mortgage', 'paint']

events['auto_related'] = detect_keyword_flag(events, category_candidates, auto_keywords)  # авто-расходы/интерес
    
events['home_related'] = detect_keyword_flag(events, category_candidates, home_keywords)  # ремонт/недвижимость
print('Доля авто-событий:', events['auto_related'].mean() if len(events) else 0)
print('Доля ремонт/недвижимость событий:', events['home_related'].mean() if len(events) else 0)
print(events[['auto_related', 'home_related']].head())


Доля авто-событий: 0.0
Доля ремонт/недвижимость событий: 0.0
     auto_related  home_related
61          False         False
146         False         False
165         False         False
167         False         False
168         False         False


In [12]:
# daily_amount: сумма трат за день; daily_events: число событий за день; unique_categories: разнообразие
agg_dict = {
    'daily_amount': ('amount', 'sum'),
    'daily_events': ('event_dt', 'size'),
}
if category_col:
    agg_dict['unique_categories'] = (category_col, 'nunique')

daily = events.groupby(['user_id', 'date']).agg(**agg_dict).reset_index() if len(events) else pd.DataFrame()
if len(daily):
    daily = daily.sort_values(['user_id', 'date'])
print('Размер daily:', daily.shape)
print(daily.head())

Размер daily: (33377, 5)
   user_id       date  daily_amount  daily_events  unique_categories
0     1421 2025-09-27           0.0            12                  1
1     2446 2025-09-27           0.0             4                  3
2     7048 2025-09-27           0.0             2                  1
3    21317 2025-09-27           0.0             3                  1
4    23752 2025-09-27           0.0            11                  4


In [13]:
# top_category: самая дорогая категория дня (сумма/частота)
if len(daily) and category_col:
    cat_daily = (
        events
        .groupby(['user_id', 'date', category_col])
        .agg(category_amount=('amount', 'sum'), category_events=('amount', 'size'))
        .reset_index()
    )
    top_cat = (
        cat_daily
        .sort_values(['category_amount', 'category_events'], ascending=False)
        .groupby(['user_id', 'date'])
        .head(1)
        .rename(columns={category_col: 'top_category'})
    )
    daily = daily.merge(top_cat[['user_id', 'date', 'top_category', 'category_amount', 'category_events']], on=['user_id', 'date'], how='left')
    print('Добавлен top_category')
elif not category_col:
    print('Нет колонки категории, пропускаем top_category')
else:
    print('daily пустой, top_category пропущен')

print(daily.head())

Добавлен top_category
   user_id       date  daily_amount  daily_events  unique_categories  \
0     1421 2025-09-27           0.0            12                  1   
1     2446 2025-09-27           0.0             4                  3   
2     7048 2025-09-27           0.0             2                  1   
3    21317 2025-09-27           0.0             3                  1   
4    23752 2025-09-27           0.0            11                  4   

  top_category  category_amount  category_events  
0       search              0.0               12  
1          u2i              0.0                2  
2      catalog              0.0                2  
3       search              0.0                3  
4          i2i              0.0                4  


In [14]:
# был ли в этот день хотя бы один авто/ремонт-событие
for source_col, target_col in [('auto_related', 'is_auto_active'), ('home_related', 'is_home_repair_period')]:
    if source_col in events.columns and len(events):
        flag = events.groupby(['user_id', 'date'])[source_col].any().reset_index().rename(columns={source_col: target_col})
        daily = daily.merge(flag, on=['user_id', 'date'], how='left') if len(daily) else flag
    else:
        daily[target_col] = False
for col in ['is_auto_active', 'is_home_repair_period']:
    if col in daily.columns:
        daily[col] = daily[col].fillna(False)
print('Поля is_auto_active / is_home_repair_period добавлены')
print(daily[['is_auto_active', 'is_home_repair_period']].head())


Поля is_auto_active / is_home_repair_period добавлены
   is_auto_active  is_home_repair_period
0           False                  False
1           False                  False
2           False                  False
3           False                  False
4           False                  False


In [15]:
# is_high_spend_day: траты >= 75 перцентиля для этого пользователя
# saver_share_30d: доля низких трат в окне 15 дней; is_saver = стабильный накопитель
if len(daily):
    p75 = daily.groupby('user_id')['daily_amount'].transform(lambda s: s.quantile(0.75))
    daily['is_high_spend_day'] = daily['daily_amount'] >= p75

    low_spend_flag = daily['daily_amount'] <= daily.groupby('user_id')['daily_amount'].transform('median')
    daily['saver_share_15d'] = low_spend_flag.groupby(daily['user_id']).transform(lambda s: s.rolling(30, min_periods=10).mean())
    daily['is_saver'] = daily['saver_share_15d'] >= 0.8
else:
    daily['is_high_spend_day'] = []
    daily['saver_share_15d'] = []
    daily['is_saver'] = []

In [16]:
daily[['daily_amount', 'is_high_spend_day', 'saver_share_15d', 'is_saver']].head()

,daily_amount,is_high_spend_day,saver_share_15d,is_saver
0,0.0,True,NaN,False
1,0.0,True,NaN,False
2,0.0,True,NaN,False
3,0.0,True,NaN,False
4,0.0,True,NaN,False


In [17]:
def build_calendar_flags(dates: pd.Series) -> pd.DataFrame:
    d = pd.to_datetime(dates)
    day = d.dt.day
    month = d.dt.month
    flags = pd.DataFrame(index=dates.index)
    flags['is_pre_new_year'] = (month == 12) & (day >= 15)  # 15–31 декабря
    flags['is_gifts_q1'] = (month == 2) | ((month == 3) & (day <= 8))  # февраль–8 марта
    flags['is_back_to_school'] = ((month == 8) & (day >= 15)) | ((month == 9) & (day <= 15))  # школа/учеба
    flags['is_summer'] = month.isin([6, 7, 8])  # лето / турсезон
    flags['is_salary_window'] = (day >= 20) | (day <= 5)  # зарплатное окно
    flags['is_social_benefits_window'] = (day >= 10) & (day <= 20)  # выплаты от государства
    return flags

In [18]:
# Календарные признаки: сезонные флаги + базовые поля даты
if len(daily):
    calendar_flags = build_calendar_flags(daily['date'])
    daily = pd.concat([daily, calendar_flags], axis=1)
    daily['month'] = pd.to_datetime(daily['date']).dt.month
    daily['dayofweek'] = pd.to_datetime(daily['date']).dt.dayofweek
    daily['is_weekend'] = daily['dayofweek'] >= 5
    daily['weekofyear'] = pd.to_datetime(daily['date']).dt.isocalendar().week.astype(int)
else:
    daily['month'] = []
    daily['dayofweek'] = []
    daily['is_weekend'] = []
    daily['weekofyear'] = []
daily[['date', 'month', 'weekofyear', 'dayofweek', 'is_weekend']].head()

,date,month,weekofyear,dayofweek,is_weekend
0,2025-09-27,9,39,5,True
1,2025-09-27,9,39,5,True
2,2025-09-27,9,39,5,True
3,2025-09-27,9,39,5,True
4,2025-09-27,9,39,5,True


# Демография

In [19]:
if USERS_PATH.exists() and len(daily):
    users = pd.read_parquet(USERS_PATH)
    daily = daily.merge(users, on='user_id', how='left')
    print('Добавили демографию', users.columns.tolist())
elif not USERS_PATH.exists():
    print('users.pq не найден, пропускаем демографию')
else:
    print('daily пустой, демографию не добавляем')
daily.head()

Добавили демографию ['user_id', 'socdem_cluster', 'region']


,user_id,date,daily_amount,daily_events,unique_categories,top_category,category_amount,category_events,is_auto_active,is_home_repair_period,...,is_back_to_school,is_summer,is_salary_window,is_social_benefits_window,month,dayofweek,is_weekend,weekofyear,socdem_cluster,region
0,1421,2025-09-27,0.0,12,1,search,0.0,12,False,False,...,False,False,True,False,9,5,True,39,20.0,59.0
1,2446,2025-09-27,0.0,4,3,u2i,0.0,2,False,False,...,False,False,True,False,9,5,True,39,5.0,61.0
2,7048,2025-09-27,0.0,2,1,catalog,0.0,2,False,False,...,False,False,True,False,9,5,True,39,17.0,60.0
3,21317,2025-09-27,0.0,3,1,search,0.0,3,False,False,...,False,False,True,False,9,5,True,39,12.0,38.0
4,23752,2025-09-27,0.0,11,4,i2i,0.0,4,False,False,...,False,False,True,False,9,5,True,39,7.0,76.0


In [20]:
def apply_feature_product_mapping(df: pd.DataFrame, feature_to_types: dict, product_types: list) -> pd.DataFrame:
    targets = pd.DataFrame(index=df.index)
    for pt in product_types:
        targets[f'target_{pt}'] = False
    for feat, pts in feature_to_types.items():
        if feat not in df.columns:
            continue
        for pt in pts:
            if pt in product_types:
                targets[f'target_{pt}'] = targets[f'target_{pt}'] | df[feat].fillna(False)
    target_cols = [f'target_{pt}' for pt in product_types]
    targets['candidate_product_types'] = targets[target_cols].apply(lambda r: [pt for pt, flag in zip(product_types, r) if flag], axis=1)
    return targets


In [21]:
with open(PRODUCTS_PATH, 'r') as f:
    products = json.load(f)
product_types = sorted({p['product_type'] for p in products})

# какая фича сигналит в пользу какого типа продукта
feature_to_types = {
    'is_pre_new_year': ['loan', 'credit_card'],
    'is_gifts_q1': ['credit_card', 'debit_card', 'premium_service'],
    'is_back_to_school': ['credit_card', 'loan', 'savings_account', 'deposit'],
    'is_summer': ['insurance', 'debit_card'],
    'is_salary_window': ['debit_card', 'deposit'],
    'is_social_benefits_window': ['savings_account', 'deposit'],
    'is_high_spend_day': ['loan', 'credit_card'],
    'is_saver': ['deposit', 'savings_account', 'investment'],
    'is_auto_active': ['insurance', 'debit_card'],
    'is_home_repair_period': ['mortgage', 'loan', 'insurance'],
}

targets = apply_feature_product_mapping(daily, feature_to_types, product_types)
daily = pd.concat([daily, targets], axis=1)
print('Типы продуктов:', product_types)
daily.filter(regex='^target_').head()

Типы продуктов: ['card_accessory', 'credit_card', 'debit_card', 'deposit', 'insurance', 'investment', 'investment_education', 'investment_ideas', 'loan', 'mortgage', 'partner_program', 'premium_service', 'savings_account']


,target_card_accessory,target_credit_card,target_debit_card,target_deposit,target_insurance,target_investment,target_investment_education,target_investment_ideas,target_loan,target_mortgage,target_partner_program,target_premium_service,target_savings_account
0,False,True,True,True,False,False,False,False,True,False,False,False,False
1,False,True,True,True,False,False,False,False,True,False,False,False,False
2,False,True,True,True,False,False,False,False,True,False,False,False,False
3,False,True,True,True,False,False,False,False,True,False,False,False,False
4,False,True,True,True,False,False,False,False,True,False,False,False,False


In [22]:
final_cols = ['user_id', 'date', 'month', 'weekofyear', 'dayofweek', 'is_weekend', 'daily_amount', 'daily_events']
for col in ['unique_categories', 'top_category', 'category_amount', 'category_events', 'is_auto_active', 'is_home_repair_period', 'is_high_spend_day', 'is_saver', 'saver_share_30d', 'is_pre_new_year', 'is_gifts_q1', 'is_back_to_school', 'is_summer', 'is_salary_window', 'is_social_benefits_window']:
    if col in daily.columns:
        final_cols.append(col)
product_cols = [col for col in daily.columns if col.startswith('target_')]
final_cols.extend(product_cols)
if 'candidate_product_types' in daily.columns:
    final_cols.append('candidate_product_types')

final_df = daily[final_cols].sort_values(['user_id', 'date']) if len(daily) else daily.copy()
print('Финальный датафрейм:', final_df.shape)
final_df.head()

Финальный датафрейм: (33377, 36)


,user_id,date,month,weekofyear,dayofweek,is_weekend,daily_amount,daily_events,unique_categories,top_category,...,target_insurance,target_investment,target_investment_education,target_investment_ideas,target_loan,target_mortgage,target_partner_program,target_premium_service,target_savings_account,candidate_product_types
0,1421,2025-09-27,9,39,5,True,0.0,12,1,search,...,False,False,False,False,True,False,False,False,False,"[credit_card, debit_card, deposit, loan]"
1,2446,2025-09-27,9,39,5,True,0.0,4,3,u2i,...,False,False,False,False,True,False,False,False,False,"[credit_card, debit_card, deposit, loan]"
2,7048,2025-09-27,9,39,5,True,0.0,2,1,catalog,...,False,False,False,False,True,False,False,False,False,"[credit_card, debit_card, deposit, loan]"
3,21317,2025-09-27,9,39,5,True,0.0,3,1,search,...,False,False,False,False,True,False,False,False,False,"[credit_card, debit_card, deposit, loan]"
4,23752,2025-09-27,9,39,5,True,0.0,11,4,i2i,...,False,False,False,False,True,False,False,False,False,"[credit_card, debit_card, deposit, loan]"
